In [2]:
import pandas as pd 
import numpy as np 

df = pd.read_csv('../data/raw/diabetes.csv')

print(f"Original dataset shape: {df.shape}")

Original dataset shape: (768, 9)


In [5]:
# Data quality checks 
# Columns where 0 is not possible 

zero_not_allowed = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

quality_report = {}

for col in zero_not_allowed:
    zero_count = (df[col] == 0).sum()
    zero_pct = (zero_count /len(df)) * 100
    quality_report[col] = {
        'zero_count': zero_count,
        'zero_pct' : round(zero_pct, 2) 
    }

quality_df = pd.DataFrame(quality_report).T
print('\n' + '='*30)
print('Invalid zero values')
print(quality_df)

total_invalid = df[zero_not_allowed].eq(0).any(axis=1).sum()
print(f'\n Records with at least one invalid zero: {total_invalid}')



Invalid zero values
               zero_count  zero_pct
Glucose               5.0      0.65
BloodPressure        35.0      4.56
SkinThickness       227.0     29.56
Insulin             374.0     48.70
BMI                  11.0      1.43

 Records with at least one invalid zero: 376


In [7]:
df_cleaned = df.copy() 

for col in zero_not_allowed:
    df_cleaned[col] = df_cleaned[col].replace(0, np.nan)

# Missing values after zero replacement 

print(df_cleaned.isnull().sum())
print(f'Total missing values: {df_cleaned.isnull().sum().sum()}')


Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64
Total missing values: 652


In [8]:
# Imputation - by median 

imputation_log = {}

for col in zero_not_allowed:
    missing_count = df_cleaned[col].isnull().sum()

    if missing_count > 0:
        median_val = df_cleaned[col].median()

        df_cleaned[col] = df_cleaned[col].fillna(median_val)

        imputation_log[col] = {
            'method' : 'median',
            'value' : round(median_val, 2),
            'count_imputed' : missing_count
        }


imputation_df = pd.DataFrame(imputation_log).T

print('Imputation Log')
print(imputation_df)

print(f"Remaining missing value: {df_cleaned.isnull().sum().sum()}")


Imputation Log
               method  value count_imputed
Glucose        median  117.0             5
BloodPressure  median   72.0            35
SkinThickness  median   29.0           227
Insulin        median  125.0           374
BMI            median   32.3            11
Remaining missing value: 0


In [9]:
# Healthcare range checks 

def validate_ranges(df):

    validation_results = {}

    ranges = {
        'Glucose': (0, 300),          # mg/dL
        'BloodPressure': (30, 200),   # mmHg
        'SkinThickness': (0, 100),    # mm
        'Insulin': (0, 900),          # μU/mL
        'BMI': (10, 70),              # kg/m²
        'Age': (0, 120)               # years
    }

    for col, (min_val, max_val) in ranges.items():
        mask = (df[col] < min_val) | (df[col] > max_val) 
        outliers_count = mask.sum()

        validation_results[col] = {
            'outliers_count' : outliers_count, 
            'outliers_pct' : round(outliers_count / len(df) * 100, 2),
            'valid_range' : f"{min_val}-{max_val}"

        }


    return validation_results 

validation_report = validate_ranges(df_cleaned) 
validation_df = pd.DataFrame(validation_report).T


print("=" * 30)
print("Data Validation Report - Range Checks")
print(validation_df) 
    

Data Validation Report - Range Checks
              outliers_count outliers_pct valid_range
Glucose                    0          0.0       0-300
BloodPressure              1         0.13      30-200
SkinThickness              0          0.0       0-100
Insulin                    0          0.0       0-900
BMI                        0          0.0       10-70
Age                        0          0.0       0-120


**Note:** Currently only 1 outlier detected (BloodPressure), so capping does not materially change the data.

In [10]:

output_path = '../data/processed/diabetes_cleaned.csv'
df_cleaned.to_csv(output_path, index=False)


print("DATA CLEANING COMPLETE")
print("="*60)
print(f"Original shape: {df.shape}")
print(f"Cleaned shape:  {df_cleaned.shape}")
print(f"Records preserved: {len(df_cleaned)/len(df)*100:.1f}%")
print(f"\nSaved to: {output_path}")



DATA CLEANING COMPLETE
Original shape: (768, 9)
Cleaned shape:  (768, 9)
Records preserved: 100.0%

Saved to: ../data/processed/diabetes_cleaned.csv


In [11]:
# Quick stats comparison
print("\n" + "="*60)
print("BEFORE vs AFTER - Key Metrics")
print("="*60)
comparison = pd.DataFrame({
    'Before': df[zero_not_allowed].describe().loc['mean'],
    'After': df_cleaned[zero_not_allowed].describe().loc['mean']
})
print(comparison.round(2))



BEFORE vs AFTER - Key Metrics
               Before   After
Glucose        120.89  121.66
BloodPressure   69.11   72.39
SkinThickness   20.54   29.11
Insulin         79.80  140.67
BMI             31.99   32.46


In [12]:
import json

lineage_doc = {
    "source_file": "diabetes.csv",
    "source_records": len(df),
    "cleaning_date": pd.Timestamp.now().strftime('%Y-%m-%d %H:%M'),
    "transformations": [
        "Replaced invalid zeros with NaN in 5 medical columns",
        "Applied median imputation for missing values",
        "Validated ranges against clinical standards"
    ],
    "quality_metrics": {
        "records_processed": len(df_cleaned),
        "records_preserved": len(df_cleaned),
        "invalid_zeros_handled": int(quality_df['zero_count'].sum())
    },
    "output_file": "diabetes_cleaned.csv"
}

with open('../data/processed/cleaning_lineage.json', 'w') as f:
    json.dump(lineage_doc, f, indent=4)

Data Cleaning & Lineage Summary

**Source dataset:** `data/raw/diabetes.csv`  
**Records processed:** 768  
**Records preserved:** 768 (100%)  

## Transformations Applied
1. Replaced invalid zeros with NaN in 5 medical columns: `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin`, `BMI`
2. Applied **median imputation** for missing values
3. Validated ranges against clinical standards

## Quality Metrics
- **Invalid zeros handled:** 652   
- **Dataset shape before cleaning:** (768, 9)  
- **Dataset shape after cleaning:** (768, 9)  